In [7]:
# imports
import requests
import json
from datetime import datetime

In [58]:
# setup
call_id = "f4ea020d-1e2d-45b1-af9d-02d42a6faf21"
BLEND_API_KEY = "org_ba2e4ccfb75e56afc088d9804df57d2623542e8bbd3de2c02bfcb0024daa778c1850bba9de94a2d1ec6a69"
GROQ_API_KEY = "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"
url = "https://api.bland.ai/v1/calls"
groq_url = "https://api.groq.com/openai/v1/chat/completions"

In [59]:
def get_follow_up_date_groq(transcription, call_date):
    # Call to Groq API
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "deepseek-r1-distill-llama-70b",
        "messages": [
            {"role": "system", "content": "You are an AI assistant tasked with analyzing call transcripts and suggesting specific follow-up dates, please resonse with specific time and date format, for example: \"February 6, 2025 12:00 AM\", do not add any other text"},
            {"role": "user", "content": f"Based on this call transcript, suggest a specific follow-up date: {transcription}, the time of the call is{call_date}"}
        ]
    }
    groq_response = requests.post(groq_url, json=payload, headers=headers)
    return groq_response.json()['choices'][0]['message']['content'].strip().split('\n')[-1]
    
    
    
    


In [60]:
#pull call information

# Headers
headers = {
   'Authorization': BLEND_API_KEY,
   "Content-Type": "application/json"
}

response = requests.request("GET", url+"/"+call_id, headers=headers)
for line in response.text.split(","):
    print(line)
    


{"call_id":"f4ea020d-1e2d-45b1-af9d-02d42a6faf21"
"c_id":"f4ea020d-1e2d-45b1-af9d-02d42a6faf21"
"call_length":2.91666666666667
"batch_id":null
"to":"6a5a0412-6481-4533-b560-cf72283e956b"
"from":null
"sid":"CAmb0oajpbnhar4o44z138j"
"completed":true
"created_at":"2025-02-10T08:07:15.999+00:00"
"inbound":true
"queue_status":"complete"
"max_duration":30
"error_message":null
"variables":{"customer name":"John"
"home address":"123 Main St"
"agent name":"Christin"
"language":"English"
"random_uuid":"0f57650c-1c7c-4859-a594-97452ccb1fab"
"call_id":"f4ea020d-1e2d-45b1-af9d-02d42a6faf21"
"now":"Monday
 February 10
 2025 12:07 AM"
"now_utc":"Monday
 February 10
 2025 8:07 AM"
"timestamp":"2025-02-10T00:07:16-08:00"
"timezone":"America/Los_Angeles"
"callID":"f4ea020d-1e2d-45b1-af9d-02d42a6faf21"
"close call date":null
"email":"john@gmail.com"}
"answered_by":null
"record":false
"recording_url":null
"twilio_account_sid":null
"metadata":null
"summary":"Here is a concise and insightful summary of the 

In [61]:
response_json = json.loads(response.text)

if "Interested" in response_json["pathway_tags"]:
    print(response_json["variables"]["email"])
    print(response_json["created_at"])

    follow_up_date = get_follow_up_date_groq(response_json['concatenated_transcript'], response_json["created_at"])
    
    print(follow_up_date)
        
        
    print(response_json["pathway_tags"])
    print(response_json["concatenated_transcript"])

john@gmail.com
2025-02-10T08:07:15.999+00:00
February 11, 2025 09:00 AM
['Interested']
assistant: Hi, is this John? 
 assistant: Hi, is this John? 
 user: Yes. So 
 assistant: Hi J- my name is Christin and I'm calling from TrueClaim Services. We specialize in auditing public records to search for unclaimed funds held by government agencies and owed to individuals. I believe I've found some funds that might belong to you. Can you confirm if you used to live at 123 Main St before? 
 user: Yes. 
 assistant: It appears that the local courts are holding onto some funds that belong to you, likely as a result of a foreclosure. If these funds aren't claimed, they'll eventually go to the government. We can help you recover them. Our service takes care of everything, from preparing and submitting the necessary paperwork to dealing with the bureaucracy, all at no upfront cost to you. We only get paid if you receive your money back. Would you like to learn more about the process and potentially re